# Crimes in Chicago 

Dati presi da Kaggle https://www.kaggle.com/currie32/crimes-in-chicago .
L'obiettivo del notebbok è quello di eseguire delle analisi esplorative sui crimini commessi a Chicago in un orizzonte temporale di 10 anni, dal 2007 al 2017 e di predire il numero di crimini nel 2018.

Iniziamo importando le librerie python che verrano utilizzate all'interno del documento 

In [1]:
#analisi EDA   
import pandas as pd 
import numpy as np 

#accesso alle funzionalità del sistema operativo 
import os 

print ("Setup Complete.")

Setup Complete.


In [2]:
#codice per il reload automatico e inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Dichiariamo ora le costanti.

In [3]:
BASEPATH = os.path.abspath('')

DATASET1 = 'Chicago_Crimes_2005_to_2007.csv'
DATASET2 = 'Chicago_Crimes_2008_to_2011.csv'
DATASET3 = 'Chicago_Crimes_2012_to_2017.csv'

PATH1 = os.path.join(BASEPATH,DATASET1)
PATH2 = os.path.join(BASEPATH,DATASET2)
PATH3 = os.path.join(BASEPATH,DATASET3)

print("Path 1:",PATH1)
print("Path 2:",PATH2)
print("Path 3:",PATH3)

Path 1: C:\Users\nigroch1\Documents\DATA SCIENCE COURSE\Esame_Corso_DataScience\Chicago_Crimes_2005_to_2007.csv
Path 2: C:\Users\nigroch1\Documents\DATA SCIENCE COURSE\Esame_Corso_DataScience\Chicago_Crimes_2008_to_2011.csv
Path 3: C:\Users\nigroch1\Documents\DATA SCIENCE COURSE\Esame_Corso_DataScience\Chicago_Crimes_2012_to_2017.csv


### Importazione dei dati

Importo i tre csv separatamente e li concateno in modo tale da creare un'unico dataframe contenente i dati dal 2007 al 2017. Prima di concatenarli però effettuo pulizia dei dati sui singoli dataframe (questo per permettere migliore sostituzione di eventuali valori NaN)

#### Crimini relativi all'anno 2007

In [4]:
#importo il csv sulle analisi dei crimini dal 2005 al 2007
Chicago_05_07 = pd.read_csv(PATH1, header=0, error_bad_lines = False)
Chicago_05_07 = Chicago_05_07.iloc[:,1:]
Chicago_05_07.head()

b'Skipping line 533719: expected 23 fields, saw 24\n'


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,4673626,HM274058,04/02/2006 01:00:00 PM,055XX N MANGO AVE,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,...,45.0,11.0,26,1136872.0,1936499.0,2006,04/15/2016 08:55:02 AM,41.981913,-87.771996,"(41.981912692, -87.771996382)"
1,4673627,HM202199,02/26/2006 01:40:48 PM,065XX S RHODES AVE,2017,NARCOTICS,MANU/DELIVER:CRACK,SIDEWALK,True,False,...,20.0,42.0,18,1181027.0,1861693.0,2006,04/15/2016 08:55:02 AM,41.775733,-87.611920,"(41.775732538, -87.611919814)"
2,4673628,HM113861,01/08/2006 11:16:00 PM,013XX E 69TH ST,051A,ASSAULT,AGGRAVATED: HANDGUN,OTHER,False,False,...,5.0,69.0,04A,1186023.0,1859609.0,2006,04/15/2016 08:55:02 AM,41.769897,-87.593671,"(41.769897392, -87.593670899)"
3,4673629,HM274049,04/05/2006 06:45:00 PM,061XX W NEWPORT AVE,0460,BATTERY,SIMPLE,RESIDENCE,False,False,...,38.0,17.0,08B,1134772.0,1922299.0,2006,04/15/2016 08:55:02 AM,41.942984,-87.780057,"(41.942984005, -87.780056951)"
4,4673630,HM187120,02/17/2006 09:03:14 PM,037XX W 60TH ST,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,ALLEY,True,False,...,13.0,65.0,18,1152412.0,1864560.0,2006,04/15/2016 08:55:02 AM,41.784211,-87.716745,"(41.784210853, -87.71674491)"


In [5]:
print("Il dataframe 'Chicago_05_07' contiene:\n",format(Chicago_05_07.shape[0],',d'),"righe\n",Chicago_05_07.shape[1],"colonne")

Il dataframe 'Chicago_05_07' contiene:
 1,872,343 righe
 22 colonne


In [6]:
#che tipo di dari ho? 
Chicago_05_07.dtypes

ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object

Come prima cosa, per lavorare agevolmente con il dataframe, rinomino le colonne in modo tale da eliminare lo spazio nel nome e trasformo nel formato corretto i campi contenenti una data.

In [7]:
#funzione replace
Chicago_05_07.columns=[c.replace( ' ', '_') for c in Chicago_05_07.columns]
Chicago_05_07.columns

Index(['ID', 'Case_Number', 'Date', 'Block', 'IUCR', 'Primary_Type',
       'Description', 'Location_Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community_Area', 'FBI_Code', 'X_Coordinate',
       'Y_Coordinate', 'Year', 'Updated_On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [8]:
#trasformo i campi 'Date' e 'Update_On' da object a datetime64
Chicago_05_07.Date = pd.to_datetime(Chicago_05_07.Date, format='%m/%d/%Y %I:%M:%S %p')
Chicago_05_07.Updated_On = pd.to_datetime(Chicago_05_07.Updated_On, format='%m/%d/%Y %I:%M:%S %p')
Chicago_05_07.dtypes

ID                               int64
Case_Number                     object
Date                    datetime64[ns]
Block                           object
IUCR                            object
Primary_Type                    object
Description                     object
Location_Description            object
Arrest                            bool
Domestic                          bool
Beat                             int64
District                       float64
Ward                           float64
Community_Area                 float64
FBI_Code                        object
X_Coordinate                   float64
Y_Coordinate                   float64
Year                             int64
Updated_On              datetime64[ns]
Latitude                       float64
Longitude                      float64
Location                        object
dtype: object

##### Trattamento valori duplicati

Controllo che il dataframe non contenga righe duplicate e procedo all'analisi dei valori NaN.

In [9]:
#controllo se ci sono valori duplicati
Chicago_05_07.duplicated().sum()

533716

In [10]:
Chicago_05_07.loc[Chicago_05_07.duplicated()==True]

,ID,Case_Number,Date,Block,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,...,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Updated_On,Latitude,Longitude,Location
533717,4673627,HM202199,2006-02-26 13:40:48,065XX S RHODES AVE,2017,NARCOTICS,MANU/DELIVER:CRACK,SIDEWALK,True,False,...,20.0,42.0,18,1181027.0,1861693.0,2006,2016-04-15 08:55:02,41.775733,-87.611920,"(41.775732538, -87.611919814)"
533718,4673628,HM113861,2006-01-08 23:16:00,013XX E 69TH ST,051A,ASSAULT,AGGRAVATED: HANDGUN,OTHER,False,False,...,5.0,69.0,04A,1186023.0,1859609.0,2006,2016-04-15 08:55:02,41.769897,-87.593671,"(41.769897392, -87.593670899)"
533719,4673629,HM274049,2006-04-05 18:45:00,061XX W NEWPORT AVE,0460,BATTERY,SIMPLE,RESIDENCE,False,False,...,38.0,17.0,08B,1134772.0,1922299.0,2006,2016-04-15 08:55:02,41.942984,-87.780057,"(41.942984005, -87.780056951)"
533720,4673630,HM187120,2006-02-17 21:03:14,037XX W 60TH ST,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,ALLEY,True,False,...,13.0,65.0,18,1152412.0,1864560.0,2006,2016-04-15 08:55:02,41.784211,-87.716745,"(41.784210853, -87.71674491)"
533721,4673631,HM263167,2006-03-30 22:30:00,014XX W 73RD PL,0560,ASSAULT,SIMPLE,APARTMENT,True,False,...,17.0,67.0,08A,1167688.0,1855998.0,2006,2016-04-15 08:55:02,41.760401,-87.660982,"(41.760401372, -87.660982392)"
533722,4673632,HM273234,2006-04-05 12:10:00,050XX N LARAMIE AVE,0460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",True,False,...,45.0,11.0,08B,1140791.0,1932993.0,2006,2016-04-15 08:55:02,41.972221,-87.757670,"(41.972220564, -87.75766982)"
533723,4673633,HM275105,2006-04-05 15:00:00,067XX S ROCKWELL ST,0820,THEFT,$500 AND UNDER,STREET,False,False,...,15.0,66.0,06,1160205.0,1859776.0,2006,2016-04-15 08:55:02,41.770926,-87.688304,"(41.770925978, -87.688304107)"
533724,4673634,HM275063,2006-04-05 21:30:00,019XX W CHICAGO AVE,0560,ASSAULT,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),False,False,...,32.0,24.0,08A,1163122.0,1905349.0,2006,2016-04-15 08:55:02,41.895923,-87.676334,"(41.895922672, -87.676333733)"
533725,4673635,HM268513,2006-04-03 03:00:00,063XX S EBERHART AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,SIDEWALK,False,True,...,20.0,42.0,08B,1180669.0,1863047.0,2006,2016-04-15 08:55:02,41.779456,-87.613191,"(41.77945628, -87.613190628)"
533726,4673637,HM275073,2006-04-06 11:15:00,0000X N LA SALLE ST,0810,THEFT,OVER $500,STREET,False,False,...,42.0,32.0,06,1175135.0,1900412.0,2006,2016-04-15 08:55:02,41.882114,-87.632361,"(41.882114362, -87.632361012)"


In [11]:
#elimino righe duplicate 
x=Chicago_05_07.duplicated().sum()
Chicago_05_07 = Chicago_05_07.drop_duplicates()
print("Sono stati eliminati",x,"record.")
print("Il dataframe 'Chicago_05_07' ora contiene:\n",format(Chicago_05_07.shape[0],',d'),"righe\n",Chicago_05_07.shape[1],"colonne")

Sono stati eliminati 533716 record.
Il dataframe 'Chicago_05_07' ora contiene:
 1,338,627 righe
 22 colonne


##### Trattamento valori nulli 

In [12]:
#controllo numero dei valori null
Chicago_05_07.isnull().sum()

ID                         0
Case_Number                0
Date                       0
Block                      0
IUCR                       0
Primary_Type               0
Description                0
Location_Description      20
Arrest                     0
Domestic                   0
Beat                       0
District                   4
Ward                       9
Community_Area           261
FBI_Code                   0
X_Coordinate            7471
Y_Coordinate            7471
Year                       0
Updated_On                 0
Latitude                7471
Longitude               7471
Location                7471
dtype: int64

Di questo dataframe, mi servono solo i valori relativi all'anno 2007, quindi, visto che il campo Year non contiene valori nulli, eseguo la selezione del dataframe, in modo tale da valutare quanto pessano rispetto al dataframe corretto. 

In [13]:
#filtro il dataframe per year = 2007 
Chicago_05_07 = Chicago_05_07.loc[Chicago_05_07.Year == 2007]
print("Il dataframe 'Chicago_05_07' ora contiene:\n",format(Chicago_05_07.shape[0],',d'),"righe\n",Chicago_05_07.shape[1],"colonne.")

Il dataframe 'Chicago_05_07' ora contiene:
 436,924 righe
 22 colonne.


In [14]:
#controllo numero dei valori null
Chicago_05_07.isnull().sum()

ID                         0
Case_Number                0
Date                       0
Block                      0
IUCR                       0
Primary_Type               0
Description                0
Location_Description       4
Arrest                     0
Domestic                   0
Beat                       0
District                   2
Ward                       5
Community_Area           158
FBI_Code                   0
X_Coordinate            1236
Y_Coordinate            1236
Year                       0
Updated_On                 0
Latitude                1236
Longitude               1236
Location                1236
dtype: int64

In [15]:
#print la percentuale di record con valori nan rispetto al totale
print("Lo", round(Chicago_05_07.Location_Description.isnull().sum()/Chicago_05_07.shape[0]*100,4),"% dei record del campo 'Location_Description' contiene valori nulli.")
print("Lo", round(Chicago_05_07.District.isnull().sum()/Chicago_05_07.shape[0]*100,4),"% dei record del campo 'District' contiene valori nulli.")
print("Lo", round(Chicago_05_07.Ward.isnull().sum()/Chicago_05_07.shape[0]*100,4),"% dei record del campo 'Ward' contiene valori nulli.")
print("Lo", round(Chicago_05_07.Community_Area.isnull().sum()/Chicago_05_07.shape[0]*100,2),"% dei record del campo 'Community_Area' contiene valori nulli.")
print("Lo", round(Chicago_05_07.X_Coordinate.isnull().sum()/Chicago_05_07.shape[0]*100,2),"% dei record del campo 'X_Coordinate' contiene valori nulli.")
print("Lo", round(Chicago_05_07.Y_Coordinate.isnull().sum()/Chicago_05_07.shape[0]*100,2),"% dei record del campo 'Y_Coordinate' contiene valori nulli.")
print("Lo", round(Chicago_05_07.Latitude.isnull().sum()/Chicago_05_07.shape[0]*100,2),"% dei record del campo 'Latitude' contiene valori nulli.")
print("Lo", round(Chicago_05_07.Longitude.isnull().sum()/Chicago_05_07.shape[0]*100,2),"% dei record del campo 'Longitude' contiene valori nulli.")
print("Lo", round(Chicago_05_07.Location.isnull().sum()/Chicago_05_07.shape[0]*100,2),"% dei record del campo 'Location' contiene valori nulli.")

Lo 0.0009 % dei record del campo 'Location_Description' contiene valori nulli.
Lo 0.0005 % dei record del campo 'District' contiene valori nulli.
Lo 0.0011 % dei record del campo 'Ward' contiene valori nulli.
Lo 0.04 % dei record del campo 'Community_Area' contiene valori nulli.
Lo 0.28 % dei record del campo 'X_Coordinate' contiene valori nulli.
Lo 0.28 % dei record del campo 'Y_Coordinate' contiene valori nulli.
Lo 0.28 % dei record del campo 'Latitude' contiene valori nulli.
Lo 0.28 % dei record del campo 'Longitude' contiene valori nulli.
Lo 0.28 % dei record del campo 'Location' contiene valori nulli.


I record NaN sono tutti inferiori allo 0,5% del totale del dataframe. Decido quindi semplicemente di eliminare questi valori.

In [16]:
#elimino i valori NaN
Chicago_05_07= Chicago_05_07.dropna()
print("Il dataframe 'Chicago_05_07' ora contiene:\n",format(Chicago_05_07.shape[0],',d'),"righe\n",Chicago_05_07.shape[1],"colonne.")

Il dataframe 'Chicago_05_07' ora contiene:
 435,527 righe
 22 colonne.


Ora che ho pulito il mio dataframe, ordino per data descrescente e resetto l'indice

In [17]:
Chicago_05_07 = Chicago_05_07.sort_values(by=['Date']).reset_index(drop=True)
Chicago_05_07.head()

,ID,Case_Number,Date,Block,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,...,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Updated_On,Latitude,Longitude,Location
0,6023884,HP127691,2007-01-01,135XX S AVENUE K,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,...,10.0,55.0,06,1202427.0,1815919.0,2007,2016-04-15 08:55:02,41.649606,-87.535027,"(41.649605674, -87.535027004)"
1,5215251,HN101264,2007-01-01,046XX N CLARENDON AVE,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,...,46.0,3.0,26,1170082.0,1930842.0,2007,2016-04-15 08:55:02,41.965728,-87.650025,"(41.965727545, -87.650025065)"
2,6683411,HP758300,2007-01-01,034XX N SHEFFIELD AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESTAURANT,False,False,...,44.0,6.0,06,1168999.0,1922971.0,2007,2016-04-15 08:55:02,41.944153,-87.654236,"(41.944152863, -87.654236337)"
3,6680820,HP755774,2007-01-01,010XX N DRAKE AVE,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,APARTMENT,False,False,...,27.0,23.0,06,1152516.0,1906797.0,2007,2016-04-15 08:55:02,41.900112,-87.715249,"(41.900112384, -87.715249162)"
4,6672245,HP745789,2007-01-01,025XX N TALMAN AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,35.0,22.0,06,1158270.0,1916782.0,2007,2016-04-15 08:55:02,41.927396,-87.693841,"(41.927396306, -87.693841038)"


#### Crimini relativi agli anni 2008- 2011

Importo ora il secondo csv relativo ai dati dal 2008 al 2011

In [18]:
#importo il csv sulle analisi dei crimini dal 2008 al 2011
Chicago_08_11 = pd.read_csv(PATH2, header=0, error_bad_lines = False)
Chicago_08_11 = Chicago_08_11.iloc[:,1:]
Chicago_08_11.head()

b'Skipping line 1149094: expected 23 fields, saw 41\n'


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,4785,HP610824,10/07/2008 12:39:00 PM,000XX E 75TH ST,0110,HOMICIDE,FIRST DEGREE MURDER,ALLEY,True,False,...,6.0,69.0,01A,1178207.0,1855308.0,2008,08/17/2015 03:03:40 PM,41.758276,-87.622451,"(41.758275857, -87.622451031)"
1,4786,HP616595,10/09/2008 03:30:00 AM,048XX W POLK ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,24.0,25.0,01A,1144200.0,1895857.0,2008,08/17/2015 03:03:40 PM,41.870252,-87.746069,"(41.87025207, -87.746069362)"
2,4787,HP616904,10/09/2008 08:35:00 AM,030XX W MANN DR,0110,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,False,False,...,18.0,66.0,01A,1157314.0,1859778.0,2008,08/17/2015 03:03:40 PM,41.770990,-87.698901,"(41.770990476, -87.698901469)"
3,4788,HP618616,10/10/2008 02:33:00 AM,052XX W CHICAGO AVE,0110,HOMICIDE,FIRST DEGREE MURDER,RESTAURANT,False,False,...,37.0,25.0,01A,1141065.0,1904824.0,2008,08/17/2015 03:03:40 PM,41.894917,-87.757358,"(41.894916924, -87.757358147)"
4,4789,HP619020,10/10/2008 12:50:00 PM,026XX S HOMAN AVE,0110,HOMICIDE,FIRST DEGREE MURDER,GARAGE,False,True,...,22.0,30.0,01A,1154123.0,1886297.0,2008,08/17/2015 03:03:40 PM,41.843826,-87.709893,"(41.843826272, -87.709893465)"


In [19]:
print("Il dataframe 'Chicago_08_11' contiene:\n",format(Chicago_08_11.shape[0],',d'),"righe\n",Chicago_08_11.shape[1],"colonne")

Il dataframe 'Chicago_08_11' contiene:
 2,688,710 righe
 22 colonne


In [20]:
#che tipo di dati ho? 
Chicago_08_11.dtypes

ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object

In [ ]:
#funzione replace
Chicago_08_11.columns=[c.replace( ' ', '_') for c in Chicago_08_11.columns]
Chicago_08_11.columns

In [ ]:
#trasformo i campi 'Date' e 'Update_On' da object a datetime64
Chicago_08_11.Date = pd.to_datetime(Chicago_08_11.Date, format='%m/%d/%Y %I:%M:%S %p')
Chicago_08_11.Updated_On = pd.to_datetime(Chicago_08_11.Updated_On, format='%m/%d/%Y %I:%M:%S %p')
Chicago_08_11.dtypes

##### Trattamento valori duplicati

Controllo che il dataframe non contenga righe duplicate e procedo all'analisi dei valori NaN.

In [ ]:
#controllo se ci sono valori duplicati
Chicago_08_11.duplicated().sum()

In [ ]:
#elimino righe duplicate 
x=Chicago_08_11.duplicated().sum()
Chicago_08_11 = Chicago_08_11.drop_duplicates()
print("Sono stati eliminati",x,"record.")
print("Il dataframe 'Chicago_08_11' ora contiene:\n",format(Chicago_08_11.shape[0],',d'),"righe\n",Chicago_08_11.shape[1],"colonne")

##### Trattamento valori nulli 

In [ ]:
#controllo numero dei valori null
Chicago_08_11.isnull().sum()

In [ ]:
#print la percentuale di record con valori nan rispetto al totale
print("Lo", round(Chicago_08_11.Case_Number.isnull().sum()/Chicago_08_11.shape[0]*100,4),"% dei record del campo 'Case_Number' contiene valori nulli.")
print("Lo", round(Chicago_08_11.Location_Description.isnull().sum()/Chicago_08_11.shape[0]*100,4),"% dei record del campo 'Location_Description' contiene valori nulli.")
print("Lo", round(Chicago_08_11.District.isnull().sum()/Chicago_08_11.shape[0]*100,4),"% dei record del campo 'District' contiene valori nulli.")
print("Lo", round(Chicago_08_11.Ward.isnull().sum()/Chicago_08_11.shape[0]*100,4),"% dei record del campo 'Ward' contiene valori nulli.")
print("Lo", round(Chicago_08_11.Community_Area.isnull().sum()/Chicago_08_11.shape[0]*100,2),"% dei record del campo 'Community_Area' contiene valori nulli.")
print("Lo", round(Chicago_08_11.X_Coordinate.isnull().sum()/Chicago_08_11.shape[0]*100,2),"% dei record del campo 'X_Coordinate' contiene valori nulli.")
print("Lo", round(Chicago_08_11.Y_Coordinate.isnull().sum()/Chicago_08_11.shape[0]*100,2),"% dei record del campo 'Y_Coordinate' contiene valori nulli.")
print("Lo", round(Chicago_08_11.Latitude.isnull().sum()/Chicago_08_11.shape[0]*100,2),"% dei record del campo 'Latitude' contiene valori nulli.")
print("Lo", round(Chicago_08_11.Longitude.isnull().sum()/Chicago_08_11.shape[0]*100,2),"% dei record del campo 'Longitude' contiene valori nulli.")
print("Lo", round(Chicago_08_11.Location.isnull().sum()/Chicago_08_11.shape[0]*100,2),"% dei record del campo 'Location' contiene valori nulli.")

In [ ]:
#elimino i valori NaN
Chicago_08_11= Chicago_08_11.dropna()
print("Il dataframe 'Chicago_08_11' ora contiene:\n",format(Chicago_08_11.shape[0],',d'),"righe\n",Chicago_08_11.shape[1],"colonne.")

In [ ]:
Chicago_08_11 = Chicago_08_11.sort_values(by=['Date']).reset_index(drop=True)
Chicago_08_11.head()

#### Crimini relativi agli anni 2012 - 2017

Importo ora il secondo csv relativo ai dati dal 2012 al 2017

In [ ]:
#importo il csv sulle analisi dei crimini dal 2012 al 2017 
Chicago_12_17 = pd.read_csv(PATH3, header=0)
Chicago_12_17 = Chicago_12_17.iloc[:,1:]
Chicago_12_17.head() 

In [ ]:
print("Il dataframe 'Chicago_12_17' contiene:\n",format(Chicago_12_17.shape[0],',d'),"righe\n",Chicago_12_17.shape[1],"colonne")

In [ ]:
#che tipo di dati ho? 
Chicago_12_17.dtypes

In [ ]:
#funzione replace
Chicago_12_17.columns=[c.replace( ' ', '_') for c in Chicago_12_17.columns]
Chicago_12_17.columns

In [ ]:
#trasformo i campi 'Date' e 'Update_On' da object a datetime64
Chicago_12_17.Date = pd.to_datetime(Chicago_12_17.Date, format='%m/%d/%Y %I:%M:%S %p')
Chicago_12_17.Updated_On = pd.to_datetime(Chicago_12_17.Updated_On, format='%m/%d/%Y %I:%M:%S %p')
Chicago_12_17.dtypes

##### Trattamento valori duplicati

Controllo che il dataframe non contenga righe duplicate e procedo all'analisi dei valori NaN.

In [ ]:
#controllo se ci sono valori duplicati
Chicago_12_17.duplicated().sum()

In [ ]:
print("Non ci sono valori duplicati.")

##### Trattamento valori nulli 

In [ ]:
#controllo numero dei valori null
Chicago_12_17.isnull().sum()

In [ ]:
#print la percentuale di record con valori nan rispetto al totale
print("Lo", round(Chicago_12_17.Case_Number.isnull().sum()/Chicago_12_17.shape[0]*100,4),"% dei record del campo 'Case_Number' contiene valori nulli.")
print("Lo", round(Chicago_12_17.Location_Description.isnull().sum()/Chicago_12_17.shape[0]*100,4),"% dei record del campo 'Location_Description' contiene valori nulli.")
print("Lo", round(Chicago_12_17.District.isnull().sum()/Chicago_12_17.shape[0]*100,4),"% dei record del campo 'District' contiene valori nulli.")
print("Lo", round(Chicago_12_17.Ward.isnull().sum()/Chicago_12_17.shape[0]*100,4),"% dei record del campo 'Ward' contiene valori nulli.")
print("Lo", round(Chicago_12_17.Community_Area.isnull().sum()/Chicago_12_17.shape[0]*100,2),"% dei record del campo 'Community_Area' contiene valori nulli.")
print("Lo", round(Chicago_12_17.X_Coordinate.isnull().sum()/Chicago_12_17.shape[0]*100,2),"% dei record del campo 'X_Coordinate' contiene valori nulli.")
print("Lo", round(Chicago_12_17.Y_Coordinate.isnull().sum()/Chicago_12_17.shape[0]*100,2),"% dei record del campo 'Y_Coordinate' contiene valori nulli.")
print("Lo", round(Chicago_12_17.Latitude.isnull().sum()/Chicago_12_17.shape[0]*100,2),"% dei record del campo 'Latitude' contiene valori nulli.")
print("Lo", round(Chicago_12_17.Longitude.isnull().sum()/Chicago_12_17.shape[0]*100,2),"% dei record del campo 'Longitude' contiene valori nulli.")
print("Lo", round(Chicago_12_17.Location.isnull().sum()/Chicago_12_17.shape[0]*100,2),"% dei record del campo 'Location' contiene valori nulli.")

In [ ]:
#elimino i valori NaN
Chicago_12_17= Chicago_12_17.dropna()
print("Il dataframe 'Chicago_12_17' ora contiene:\n",format(Chicago_12_17.shape[0],',d'),"righe\n",Chicago_12_17.shape[1],"colonne.")

In [ ]:
Chicago_12_17 = Chicago_12_17.sort_values(by=['Date']).reset_index(drop=True)
Chicago_12_17.head()

Merge dei csv, in modo tale da avere un unico dataframe contenente i dati dal 2007 al 2017 

In [ ]:
Chicago_07_17 = pd.concat([Chicago_05_07,Chicago_08_11 , Chicago_12_17], ignore_index=False, axis=0)
Chicago_07_17.head()